# IC Pin Detection Model Training

This notebook trains a YOLOv11 model for detecting IC pins and identifying defects.

## Overview
- **Model**: YOLOv11 (You Only Look Once version 11)
- **Task**: Object detection for IC pin analysis
- **Classes**: 
  - `bent`: Bent pins (defective)
  - `okay`: Normal pins
  - `package`: IC package detection
  - `text`: Text regions on IC

## Dataset Structure
The dataset should follow YOLO format:
```
783-Pin-Detection/datasets/aug-ic-dataset/
├── images/
│   ├── train/
│   └── val/
└── labels/
    ├── train/
    └── val/
```

## 1. Install Dependencies

First, ensure you have the required packages installed.

In [ ]:
# Install required packages
# Uncomment the line below if you need to install ultralytics
# !pip install ultralytics opencv-python matplotlib pillow

## 2. Import Libraries

In [ ]:
import os
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
from PIL import Image
import yaml

## 3. Verify Dataset

Let's verify that the dataset exists and check some statistics.

In [ ]:
# Dataset paths
dataset_config = "dataset.yaml"
train_images_path = Path("783-Pin-Detection/datasets/aug-ic-dataset/images/train")
val_images_path = Path("783-Pin-Detection/datasets/aug-ic-dataset/images/val")
train_labels_path = Path("783-Pin-Detection/datasets/aug-ic-dataset/labels/train")
val_labels_path = Path("783-Pin-Detection/datasets/aug-ic-dataset/labels/val")

# Count images and labels
train_images = list(train_images_path.glob("*.jpg")) + list(train_images_path.glob("*.jpeg")) + list(train_images_path.glob("*.png"))
val_images = list(val_images_path.glob("*.jpg")) + list(val_images_path.glob("*.jpeg")) + list(val_images_path.glob("*.png"))
train_labels = list(train_labels_path.glob("*.txt"))
val_labels = list(val_labels_path.glob("*.txt"))

print(f"Training images: {len(train_images)}")
print(f"Training labels: {len(train_labels)}")
print(f"Validation images: {len(val_images)}")
print(f"Validation labels: {len(val_labels)}")

# Verify dataset config exists
if Path(dataset_config).exists():
    print(f"\n✓ Dataset configuration file '{dataset_config}' found")
    with open(dataset_config, 'r') as f:
        config = yaml.safe_load(f)
        print(f"  Classes: {config['names']}")
else:
    print(f"\n✗ Dataset configuration file '{dataset_config}' not found")

## 4. Visualize Sample Images

Let's look at some sample images from the training set.

In [ ]:
# Display a few sample training images
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, img_path in enumerate(train_images[:6]):
    img = Image.open(img_path)
    axes[i].imshow(img)
    axes[i].set_title(img_path.name)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 5. Initialize YOLO Model

We'll use a pre-trained YOLOv11 model as a starting point for transfer learning.

Available model sizes:
- `yolo11n.pt` - Nano (smallest, fastest)
- `yolo11s.pt` - Small
- `yolo11m.pt` - Medium (recommended for good balance)
- `yolo11l.pt` - Large
- `yolo11x.pt` - Extra Large (largest, most accurate)

In [ ]:
# Choose model size (change as needed)
model_size = "yolo11m.pt"  # Medium model - good balance of speed and accuracy

# Load the model
model = YOLO(model_size)
print(f"Loaded {model_size} model for transfer learning")

## 6. Configure Training Parameters

Set up the training hyperparameters. You can modify these based on your needs:

- **epochs**: Number of training iterations over the entire dataset
- **imgsz**: Input image size (higher = more accurate but slower)
- **batch**: Batch size (adjust based on GPU memory)
- **device**: GPU device to use (0 for first GPU, 'cpu' for CPU)
- **patience**: Early stopping patience (stops if no improvement)
- **cache**: Cache images in RAM for faster training

In [ ]:
# Training configuration
training_params = {
    'data': dataset_config,           # Path to dataset YAML
    'epochs': 300,                     # Number of training epochs
    'imgsz': 800,                      # Image size (pixels)
    'batch': 16,                       # Batch size (adjust based on GPU memory)
    'device': 0,                       # GPU device (0 for first GPU, 'cpu' for CPU)
    'patience': 100,                   # Early stopping patience
    'save': True,                      # Save checkpoints
    'cache': True,                     # Cache images in RAM
    'cos_lr': True,                    # Use cosine learning rate scheduler
    'close_mosaic': 10,                # Disable mosaic augmentation in last N epochs
    'box': 10,                         # Box loss weight
    'cls': 15.0,                       # Classification loss weight (higher for defect detection)
    'dfl': 3.0,                        # Distribution focal loss weight
    'flipud': 0.5,                     # Vertical flip probability
    'fliplr': 0.5,                     # Horizontal flip probability
    'degrees': 0.0,                    # Rotation degrees
    'translate': 0.1,                  # Translation
    'scale': 0.5,                      # Scale
    'project': 'runs/detect',          # Project directory
    'name': 'ic_pin_detection',        # Run name
}

print("Training Configuration:")
for key, value in training_params.items():
    print(f"  {key}: {value}")

## 7. Train the Model

Now we'll train the model. This may take a while depending on your hardware and the number of epochs.

**Note**: Training progress will be displayed, including:
- Loss values (box, cls, dfl)
- Metrics (precision, recall, mAP)
- Training time per epoch

In [ ]:
# Start training
print("Starting model training...\n")
results = model.train(**training_params)
print("\nTraining completed!")

## 8. Evaluate Model Performance

After training, evaluate the model on the validation set.

In [ ]:
# Evaluate on validation set
print("Evaluating model on validation set...\n")
metrics = model.val()

# Print key metrics
print("\nValidation Metrics:")
print(f"  Box mAP@50: {metrics.box.map50:.4f}")
print(f"  Box mAP@50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall: {metrics.box.mr:.4f}")

## 9. Visualize Training Results

Display training curves and confusion matrix.

In [ ]:
# Find the results directory
results_dir = Path(training_params['project']) / training_params['name']
print(f"Results saved to: {results_dir}")

# Display training results if they exist
results_img = results_dir / 'results.png'
confusion_matrix = results_dir / 'confusion_matrix.png'

fig, axes = plt.subplots(1, 2, figsize=(20, 8))

if results_img.exists():
    img = Image.open(results_img)
    axes[0].imshow(img)
    axes[0].set_title('Training Results')
    axes[0].axis('off')
else:
    axes[0].text(0.5, 0.5, 'Results image not found', ha='center', va='center')
    axes[0].axis('off')

if confusion_matrix.exists():
    img = Image.open(confusion_matrix)
    axes[1].imshow(img)
    axes[1].set_title('Confusion Matrix')
    axes[1].axis('off')
else:
    axes[1].text(0.5, 0.5, 'Confusion matrix not found', ha='center', va='center')
    axes[1].axis('off')

plt.tight_layout()
plt.show()

## 10. Test Model on Validation Images

Run inference on some validation images to see how the model performs.

In [ ]:
# Run inference on validation images
print("Running inference on validation images...\n")

# Select a few validation images
test_images = (list(val_images_path.glob("*.jpg")) + list(val_images_path.glob("*.jpeg")) + list(val_images_path.glob("*.png")))[:6]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, img_path in enumerate(test_images):
    # Run prediction
    results = model(str(img_path), verbose=False)
    
    # Get the annotated image
    annotated = results[0].plot()
    
    # Convert BGR to RGB for matplotlib
    import cv2
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    axes[i].imshow(annotated_rgb)
    axes[i].set_title(f"{img_path.name}")
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 11. Save and Export Model

Save the trained model in various formats for deployment.

In [ ]:
# The best model is automatically saved during training
best_model_path = results_dir / 'weights' / 'best.pt'
last_model_path = results_dir / 'weights' / 'last.pt'

print(f"Best model saved to: {best_model_path}")
print(f"Last model saved to: {last_model_path}")

# Load the best model for export
best_model = YOLO(best_model_path)

# Export to ONNX format (optional, for deployment)
# Uncomment the line below if you want to export to ONNX
# onnx_path = best_model.export(format='onnx')
# print(f"Model exported to ONNX: {onnx_path}")

## 12. Inference on New Images

Use the trained model to detect pins on new images.

In [ ]:
# Load the best trained model
trained_model = YOLO(best_model_path)

# Function to run inference on a single image
def predict_image(image_path, conf_threshold=0.25):
    """
    Run inference on an image and display results.
    
    Args:
        image_path: Path to the input image
        conf_threshold: Confidence threshold for detections (0-1)
    """
    results = trained_model(image_path, conf=conf_threshold, verbose=False)
    
    # Display results
    annotated = results[0].plot()
    import cv2
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(annotated_rgb)
    plt.title(f"Predictions on {Path(image_path).name}")
    plt.axis('off')
    plt.show()
    
    # Print detection summary
    boxes = results[0].boxes
    if len(boxes) > 0:
        print(f"\nDetected {len(boxes)} objects:")
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = trained_model.names[cls]
            print(f"  - {class_name}: {conf:.2%} confidence")
    else:
        print("\nNo objects detected")
    
    return results

# Example: Run inference on a validation image
if len(val_images) > 0:
    sample_image = val_images[0]
    print(f"Running inference on: {sample_image}\n")
    results = predict_image(sample_image, conf_threshold=0.25)

## 13. Model Summary and Next Steps

### Model Information
- The trained model can detect and classify IC pins into 4 categories: bent, okay, package, and text
- Best model weights are saved and can be loaded for inference
- Training results and metrics are available in the runs directory

### Next Steps
1. **Fine-tune hyperparameters**: Adjust learning rate, batch size, or augmentation settings
2. **Add more data**: Collect and annotate more images to improve accuracy
3. **Test on real-world data**: Validate performance on images from actual production
4. **Deploy the model**: Export to ONNX or other formats for production deployment
5. **Monitor performance**: Track model performance over time and retrain as needed

### Using the Trained Model
```python
# Load the trained model
from ultralytics import YOLO
model = YOLO('runs/detect/ic_pin_detection/weights/best.pt')

# Run inference
results = model('path/to/image.jpg')
results[0].show()
```